In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp

# Loading data

In [3]:
df_raw = pd.read_pickle('/mnt/data/mediascreen/sharedfolder/baniar_matus/em/hrns.gzip')

In [4]:
df_raw.head(15)

,hrn_source,hrn_type,name_type,prefix,first_name,middle_name,last_name,suffix,entity_name,person_unfielded,nationalities,countries_residence,dobs
hrn_id,,,,,,,,,,,,,
F_100000,FACTIVA,PERSON,PRIMARY,,Yuliia,Anatolievna,Peskovskaia,,,,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,ALIAS,,Iuliia,Anatolievna,Peskovskaya,,,Юлия Анатольевна Песковская,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,PRIMARY,,Iuliia,Anatolievna,Peskovskaya,,,,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,PRIMARY,,Yuliya,Anatolyevna,Peskovskaya,,,,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,PRIMARY,,Yulia,Anatolievna,Peskovskaya,,,,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,PRIMARY,,Iuliya,Anatolyevna,Peskovskaya,,,,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,PRIMARY,,Iulia,Anatolievna,Peskovskaya,,,,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,PRIMARY,,Iuliia,Anatolievna,Peskovskaia,,,,ru,ru,1956-07-10
F_100000,FACTIVA,PERSON,PRIMARY,,Yuliya,Anatolyevna,Peskovskaia,,,,ru,ru,1956-07-10


One specific person example

In [5]:
df_raw[df_raw.index == 'F_1006494']

,hrn_source,hrn_type,name_type,prefix,first_name,middle_name,last_name,suffix,entity_name,person_unfielded,nationalities,countries_residence,dobs
hrn_id,,,,,,,,,,,,,
F_1006494,FACTIVA,PERSON,PRIMARY,,Phuong Mai,Thi,Le,,,,vn,"ca,us","1964-00-00,1965-00-00,1966-00-00"
F_1006494,FACTIVA,PERSON,LOW_QUALITY_ALIAS,,Phuong Mai,Monica,Le,,,,vn,"ca,us","1964-00-00,1965-00-00,1966-00-00"
F_1006494,FACTIVA,PERSON,LOW_QUALITY_ALIAS,,Phuong Mai,Mai,Le,,,,vn,"ca,us","1964-00-00,1965-00-00,1966-00-00"
F_1006494,FACTIVA,PERSON,LOW_QUALITY_ALIAS,,Phuong Mai,Big Boobs,Le,,,,vn,"ca,us","1964-00-00,1965-00-00,1966-00-00"
F_1006494,FACTIVA,PERSON,LOW_QUALITY_ALIAS,,Phuong Mai,The Queen,Le,,,,vn,"ca,us","1964-00-00,1965-00-00,1966-00-00"
F_1006494,FACTIVA,PERSON,LOW_QUALITY_ALIAS,,Phuong Mai,Queen Bee,Le,,,,vn,"ca,us","1964-00-00,1965-00-00,1966-00-00"


# Basic stats

In [6]:
# no. of records
len(df_raw)

7451317

In [7]:
# no of unique ids:
len(df_raw.index.value_counts())

2702347

In [8]:
# distribution of numbers of records per 1 id
df_raw.index.value_counts().describe(percentiles=[0.25,0.5,0.75,0.9,0.95])

count    2.702347e+06
mean     2.757350e+00
std      2.847971e+00
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
90%      5.000000e+00
95%      7.000000e+00
max      3.290000e+02
Name: hrn_id, dtype: float64

$=>$ 95% or id's have 7 or less variations

### Simple dimension break downs

#### hrn_type:

In [9]:
df_raw.reset_index().groupby('hrn_type').agg({'name_type': 'count', 'hrn_id': pd.Series.nunique}).rename(columns={'name_type':'records', 'hrn_id':'unique ids'})

,records,unique ids
hrn_type,,
ENTITY,424450,163555
PERSON,7026867,2538793


#### hrn_source:

In [10]:
df_raw.reset_index().groupby('hrn_source').agg({'name_type': 'count', 'hrn_id': pd.Series.nunique}).rename(columns={'name_type':'records', 'hrn_id':'unique ids'})

,records,unique ids
hrn_source,,
FACTIVA,7451317,2702347


#### name_type:

In [11]:
df_raw.reset_index().groupby('name_type').agg({'name_type': 'count', 'hrn_id': pd.Series.nunique}).rename(columns={'name_type':'records', 'hrn_id':'unique ids'})

,records,unique ids
name_type,,
ALIAS,2455587,1256096
LOW_QUALITY_ALIAS,70163,55762
MAIDEN,69346,69084
PRIMARY,4856221,2702347


We also see that all unique IDs (2.7M) are also in PRIMARY $=>$ everyone has at least one primary record. 

# Alphabet check

In [12]:
import unicodedata as ud

latin_letters= {}

def is_latin(uchr):
    try: return latin_letters[uchr]
    except KeyError:
         return latin_letters.setdefault(uchr, 'LATIN' in ud.name(uchr))

def only_roman_chars(unistr):
    return all(is_latin(uchr)
           for uchr in unistr
           if uchr.isalpha()) # isalpha suggested by John Machin

In [13]:
only_roman_chars("hôtel lœwe")
only_roman_chars("Andrej Babiš")
only_roman_chars("Andrej Горбачёв")

True

True

False

### 1.) names

In [14]:
%%time
df_raw['name'] = df_raw['first_name'] + ' ' + df_raw['middle_name'] + ' ' + df_raw['last_name']

CPU times: user 3.1 s, sys: 2.12 s, total: 5.23 s
Wall time: 5.22 s


In [15]:
%%time
with mp.Pool(mp.cpu_count()-2) as pool:
    df_raw['name_check'] = pool.map(only_roman_chars, df_raw['name'])

CPU times: user 3.65 s, sys: 5.93 s, total: 9.58 s
Wall time: 10.3 s


In [16]:
df_raw.groupby('name_check')[['name_type']].count().rename(columns={'name_type':'records'})

,records
name_check,
False,26
True,7451291


Only 26 records in the source data have some non latin characters in one of the name field, so those are probably just some mistakes and we can ignore them.<br>
Here they are:

In [17]:
df_raw[df_raw['name_check'] == False]

,hrn_source,hrn_type,name_type,prefix,first_name,middle_name,last_name,suffix,entity_name,person_unfielded,nationalities,countries_residence,dobs,name,name_check
hrn_id,,,,,,,,,,,,,,,
F_1013763,FACTIVA,PERSON,PRIMARY,,Halaevalu,Mataʻaho,Ahomeʻe,,,,to,to,,Halaevalu Mataʻaho Ahomeʻe,False
F_1154156,FACTIVA,PERSON,PRIMARY,,Gene,С.,Geiger,,,,ud,us,,Gene С. Geiger,False
F_125465,FACTIVA,PERSON,ALIAS,,Mуkola,Mуkolayovуch,Severin,,,Микола Миколайович Северін,ua,ua,,Mуkola Mуkolayovуch Severin,False
F_125465,FACTIVA,PERSON,PRIMARY,,Mуkola,Mуkolayovуch,Severin,,,,ua,ua,,Mуkola Mуkolayovуch Severin,False
F_125604,FACTIVA,PERSON,ALIAS,,Iryna,Oleksandrivna,Smуrnova,,,Ірина Олександрівна Смирнова,ua,ua,,Iryna Oleksandrivna Smуrnova,False
F_125604,FACTIVA,PERSON,PRIMARY,,Iryna,Oleksandrivna,Smуrnova,,,,ua,ua,,Iryna Oleksandrivna Smуrnova,False
F_125813,FACTIVA,PERSON,PRIMARY,,Ludmyla,Romanivna,Людмила Романівна Маковійчук,,,,ua,ua,,Ludmyla Romanivna Людмила Романівна Маковійчук,False
F_1688697,FACTIVA,PERSON,PRIMARY,,Manuel,Mª,Cavestany Antunano,,,,uk,es,,Manuel Mª Cavestany Antunano,False
F_1688697,FACTIVA,PERSON,PRIMARY,,Manuel Mª,Mª,Cavestany Antuñano,,,,uk,es,,Manuel Mª Mª Cavestany Antuñano,False


### 2.) entities

In [18]:
%%time
with mp.Pool(mp.cpu_count()-2) as pool:
    df_raw['entity_check'] = pool.map(only_roman_chars, df_raw['entity_name'])

CPU times: user 1.89 s, sys: 5.72 s, total: 7.61 s
Wall time: 8.04 s


In [19]:
df_raw[df_raw['hrn_type'] == 'ENTITY'].groupby('entity_check')['name_type'].count()

entity_check
False     77307
True     347143
Name: name_type, dtype: int64

77K entitie records are non latin (out of 420K). Seems a lot but when we look if they all have a latin variation, then we can see that they do.

In [20]:
pd.DataFrame(df_raw[df_raw['hrn_type'] == 'ENTITY'].reset_index().groupby('hrn_id')['entity_check'].agg('max')).groupby('entity_check')['entity_check'].count()

entity_check
True    163555
Name: entity_check, dtype: int64

All 163K unique entity IDs have some latin variation in entity_name.

# Nationalities

Records without nationality:

In [21]:
len(df_raw[df_raw['nationalities'] == ''])

83094

IDs without any nationality in all records:

In [22]:
agg1 = pd.DataFrame(df_raw['nationalities'].str.len()).reset_index().groupby('hrn_id')[['nationalities']].max()
len(agg1[agg1['nationalities'] == 0])

16428

Distributions:

In [23]:
df_raw['nationalities_split'] = df_raw['nationalities'].str.split(',')

In [24]:
nat_agg = pd.DataFrame(df_raw[['nationalities_split']]\
             .explode('nationalities_split')\
             .groupby('nationalities_split')['nationalities_split']\
             .count()
            ).rename(columns={'nationalities_split':'records'}).reset_index().rename(columns={'nationalities_split':'nat'})\
    .sort_values(by='records', ascending=False)


len(nat_agg)
nat_agg.head(20)

221

,nat,records
63,es,667178
31,br,571224
168,ru,440035
205,us,433834
101,jp,311506
204,uk,285685
45,cn,257674
93,in,215474
141,mx,213891
80,gr,186269


In [25]:
print(list(nat_agg['nat']))

['es', 'br', 'ru', 'us', 'jp', 'uk', 'cn', 'in', 'mx', 'gr', 'tw', 'ua', 'pt', 'gb', 'fr', 'th', 'pe', '', 'it', 'eg', 'cl', 'pk', 'ca', 'co', 'kz', 'kr', 'ph', 'ar', 'cr', 'tr', 'id', 'do', 'uy', 'by', 'ec', 'py', 'gt', 'cy', 'my', 'sa', 'pl', 'ir', 'bd', 'iq', 'au', 'ro', 'se', 'vn', 've', 'dk', 'de', 'mm', 'bg', 'lb', 'ae', 'pa', 'ng', 'sy', 'jo', 'sd', 'za', 'sv', 'ge', 'hu', 'fi', 'nl', 'kw', 'bo', 'cu', 'RS', 'ma', 'np', 'hn', 'hr', 'be', 'af', 'no', 'sg', 'cz', 'ba', 'az', 'ke', 'dz', 'kp', 'lv', 'ao', 'ye', 'tn', 'kg', 'am', 'cd', 'ch', 'uz', 'lt', 'ly', 'il', 'ni', 'ie', 'is', 'lr', 'nz', 'bh', 'at', 'ps', 'tj', 'qa', 'ud', 'om', 'kh', 'ci', 'gh', 'sk', 'ee', 'so', 'ME', 'mr', 'si', 'lk', 'mz', 'et', 'bf', 'ug', 'tz', 'al', 'ht', 'zw', 'ml', 'tm', 'cv', 'md', 'mk', 'mn', 'bj', 'gq', 'la', 'hk', 'cm', 'mt', 'td', 'sn', 'ga', 'ne', 'sl', 'na', 'jm', 'gn', 'zm', 'cf', 'bn', 'tl', 'mg', 'gw', 'bs', 'lu', 'ad', 'mw', 'tt', 'mu', 'rw', 'bw', 'KS', 'st', 'km', 'fj', 'tg', 'gm', 'bi',

Some upper case variations.

# Countries of residence

Records without country of residence:

In [26]:
len(df_raw[df_raw['countries_residence'] == ''])

424475

IDs without any country of residence in all records:

In [27]:
agg2 = pd.DataFrame(df_raw['countries_residence'].str.len()).reset_index().groupby('hrn_id')[['countries_residence']].max()
len(agg2[agg2['countries_residence'] == 0])

163568

Distributions:

In [28]:
df_raw['countries_split'] = df_raw['countries_residence'].str.split(',')

In [29]:
country_agg = pd.DataFrame(df_raw[['countries_split']]\
             .explode('countries_split')\
             .groupby('countries_split')['countries_split']\
             .count()
            ).rename(columns={'countries_split':'records'}).reset_index().rename(columns={'countries_split':'country'})\
    .sort_values(by='records', ascending=False)


len(country_agg)
country_agg.head(20)

241

,country,records
67,es,670163
224,us,538588
32,br,519929
0,,424475
184,ru,385265
110,jp,291908
223,uk,217887
102,in,217389
151,mx,203399
47,cn,188152


In [30]:
print(list(country_agg['country']))

['es', 'us', 'br', '', 'ru', 'jp', 'uk', 'in', 'mx', 'cn', 'gr', 'tw', 'ua', 'ud', 'pt', 'gb', 'fr', 'th', 'it', 'pe', 'cl', 'eg', 'pk', 'ca', 'ph', 'hk', 'co', 'kr', 'kz', 'ar', 'cr', 'tr', 'id', 'do', 'uy', 'by', 'ec', 'au', 'py', 'my', 'gt', 'cy', 'sa', 'pl', 'se', 'bd', 'de', 'vn', 'ro', 've', 'iq', 'bg', 'ae', 'lb', 'sd', 'dk', 'mm', 'jo', 'pr', 'sy', 'ng', 'pa', 'ge', 'be', 'za', 'fi', 'hu', 'kw', 'sg', 'bo', 'np', 'RS', 'nl', 'cu', 'no', 'cz', 'hr', 'ma', 'ir', 'ke', 'ba', 'lv', 'az', 'ch', 'af', 'ao', 'dz', 'ye', 'hn', 'tn', 'am', 'cd', 'lt', 'sv', 'at', 'kp', 'ni', 'il', 'bh', 'is', 'ie', 'uz', 'kg', 'nz', 'ly', 'qa', 'kh', 'om', 'ps', 'tj', 'ci', 'gh', 'sk', 'ee', 'ME', 'mr', 'lk', 'et', 'mz', 'si', 'so', 'ug', 'bf', 'tz', 'ht', 'tm', 'ml', 'cv', 'zw', 'mk', 'mn', 'bj', 'gq', 'md', 'mo', 'la', 'lr', 'cm', 'sn', 'na', 'td', 'ga', 'gn', 'bn', 'ne', 'zm', 'jm', 'cf', 'sl', 'al', 'ad', 'va', 'lu', 'mt', 'tl', 'mg', 'gw', 'mw', 'tt', 'bw', 'mu', 'fj', 'fo', 'rw', 'bs', 'st', 'gm',

More common to have missing values. Still some upper cases.

# DOBs

Records without DOB:

In [31]:
len(df_raw[df_raw['dobs'] == ''])

3944858

IDs without any DOB in all records:

In [32]:
agg3 = pd.DataFrame(df_raw['dobs'].str.len()).reset_index().groupby('hrn_id')[['dobs']].max()
len(agg3[agg3['dobs'] == 0])

1477338